In [1]:
# Import libraries
import requests
import pandas as pd
from bokeh.plotting import output_file, show
from bokeh.models import Select, Div, Button, ColumnDataSource, HTMLTemplateFormatter,TableColumn, DataTable
from bokeh.models.widgets import DateRangePicker, DataTable, TableColumn
from bokeh.layouts import column
# Import necessary modules
from ipywidgets import interact, widgets, VBox, HBox
from IPython.display import display, HTML
from bokeh.io import push_notebook, show, output_notebook
import pandas as pd
# Import the necessary modules
from ipywidgets import HTML
from IPython.display import display, HTML as IPHTML
from datetime import datetime, timedelta

In [2]:
# Import all APIs form config.py
# WeatherAPI key
weather_key = "11da115d9a8d4556bd993522240506"

In [3]:
def get_access_token():
    # Define the URL for obtaining the access token
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    
    # Define headers for the request
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    
    # Define the data to be sent in the request body
    data = {
        "grant_type": "client_credentials",
        "client_id": 'gAJOLRiyFCjYVuCEIspShbBroNVLSgpw',  # Replace with your actual client ID
        "client_secret": 'ZDHiCX0h0NYIBm0B'  # Replace with your actual client secret
    }
 
    # Send a POST request to the specified URL with headers and data
    response = requests.post(url, headers=headers, data=data)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Extract the access token from the response JSON
        access_token = response.json()['access_token']
        print("Access Token:", access_token)  # Print the obtained access token
    else:
        # Print error message if the request was not successful
        print("Failed to obtain access token:", response.status_code, response.text)
    
    # Return the access token, if obtained successfully
    return access_token

# Get access token by calling the function
access_token = get_access_token()


Access Token: k1FXEI4d0bSZUo9dseFlEfKrYBp3


The client_id and client_secret were obtain using 'Amadeus for developers', accessed through the link:[relevant resource](https://developers.amadeus.com/self-service/apis-docs/guides/developer-guides/API-Keys/)

## Get flights according to city, airport and departure date

In [4]:
# List of cities
cities = ['Madrid', 'London', 'Paris', 'Barcelona', 'Athens']
def get_flight_details(access_token, originLocationCode, destinationLocationCode, departureDate):
    # Define the URL for the flight offers endpoint
    url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
    
    # Define headers for the request
    headers = {
        "Authorization": f'Bearer {access_token}',  # Authorization header with the access token
        "Content-Type": "application/json"  # Specify the content type as JSON
    }
    
    # Define parameters for the request
    params = {
        "originLocationCode": originLocationCode,  # Origin location code
        "destinationLocationCode": destinationLocationCode,  # Destination location code
        "departureDate": departureDate,  # Departure date
        "adults": 1,  # Number of adults
        "max": 3  # Maximum number of flight offers to retrieve
    }
    
    # Send a GET request to the specified URL with headers and parameters
    response = requests.get(url, headers=headers, params=params)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Return the JSON response containing flight details
        return response.json()
    else:
        # Print error message if the request was not successful
        print("Failed to get flight details:", response.status_code, response.text)
        return None


This function will allow to connect to the 'Amadeus' flights API and return the existing flights according to the origin and destination location and the date of the flight. This function only considers the flight for 1 adult and return the top 3 flights.

In [5]:
def process_flight_data(flight_offers_response):
    # Check if flight_offers_response is not empty
    if flight_offers_response:
        # Initialize a list to store flight data
        flight_data = []
        
        # Iterate through each flight offer in the response
        for flight in flight_offers_response['data']:
            # Extract details for the first itinerary and segment
            itinerary = flight['itineraries'][0]
            segment = itinerary['segments'][0]
            
            # Extract departure and arrival airport codes
            departure_airport = segment['departure']['iataCode']
            arrival_airport = segment['arrival']['iataCode']
            
            # Extract departure and arrival time (format: YYYY-MM-DDTHH:MM:SSZ)
            departure_time = segment['departure']['at'][-8:-3]  # Extract HH:MM part
            arrival_time = segment['arrival']['at'][-8:-3]  # Extract HH:MM part

            # Convert string time to hours and minutes
            departure_hour, departure_minute = map(int, departure_time.split(':'))
            arrival_hour, arrival_minute = map(int, arrival_time.split(':'))

            # Calculate duration in minutes
            duration_minutes = (arrival_hour * 60 + arrival_minute) - (departure_hour * 60 + departure_minute)

            # Convert duration to hours and minutes
            duration_hours = duration_minutes // 60
            duration_minutes = duration_minutes % 60

            # Format duration as HH:MM
            duration_str = f"{duration_hours:02d}:{duration_minutes:02d}"

            # Extract price and currency
            price = float(flight['price']['total'])
            currency = flight['price']['currency']
            
            # Combine price with currency
            price_with_currency = f"{price} {currency}"

            # Append flight details to flight_data list
            flight_data.append([departure_airport, arrival_airport, departure_time, arrival_time, duration_str, price_with_currency])

        # Create DataFrame from flight_data list
        df = pd.DataFrame(flight_data, columns=['DepartureAirport', 'ArrivalAirport', 'DepartureTime', 'ArrivalTime', 'Duration(hours)', 'PriceWithCurrency'])
        
        # Return the DataFrame
        return df
    else:
        # If no flight offers data is available, print a message and return None
        print("No flight offers data available.")
        return None


The previous function will return a dataframe with the information on the flights according to the criteria selected previously.

In [6]:
def get_airport_codes(city):
    # Construct the API URL to fetch airport data based on the given city
    api_url = f'https://api.api-ninjas.com/v1/airports?city={city}'
    
    # Define headers for the request, including the API key
    headers = {'X-Api-Key': 'nrc3S/XZWbfao3LeySY9hg==I1u1i9JSOP6NSPFw'}
    
    # Send a GET request to the API endpoint to fetch airport data
    response = requests.get(api_url, headers=headers)
    
    # Check if the request was successful (status code 200)
    if response.status_code == requests.codes.ok:
        # Parse the JSON response containing airport data
        airport_data = response.json()
        
        # Initialize an empty dictionary to store airport names and their corresponding IATA codes
        airport_dict = {}
        
        # Iterate over each airport in the response data
        for airport in airport_data:
            # Extract the IATA code of the airport
            airport_code = airport.get('iata')
            
            # Check if the IATA code exists (not None)
            if airport_code:
                # Extract the name of the airport
                airport_name = airport.get('name')
                
                # Add the airport name and its corresponding IATA code to the dictionary
                airport_dict[airport_name] = airport_code
        
        # Return the dictionary mapping airport names to their IATA codes
        return airport_dict
    else:
        # Print an error message if the request was not successful
        print("Error:", response.status_code, response.text)
        
        # Return an empty dictionary
        return {}


The previous function uses an API to return a dictionary with the airport code and the corresponding airport name.

In [7]:
def update_airport_options(change):
    # Extract the new value of the city selection from the change dictionary
    city = change['new']
    
    # Call the get_airport_codes function to retrieve airport codes based on the selected city
    airports = get_airport_codes(city)
    
    # Check if airport data is available for the selected city
    if airports:
        # Update the options of the airport_select dropdown widget with the retrieved airport data
        airport_select.options = airports
        
        # Set the default value of the airport_select dropdown widget to the first airport in the options
        airport_select.value = list(airports.values())[0]


The previous function allows the 'Select airport' dropdown button to change to the airport names according to the city selected.

In [46]:
# Function to update flight offers
def update_flight_offers(city, access_token, originLocationCode, destinationLocationCode, departure_date):
    '''    print("City:", city)
    print("Departure Date:", departure_date)
    print("Origin Location Code:", originLocationCode)
    print("Destination Location Code:", destinationLocationCode)'''
    
    flight_offers_response = get_flight_details(access_token, originLocationCode, destinationLocationCode, departure_date)
    #print(flight_offers_response)
    
    if flight_offers_response:
        flight_data_table = process_flight_data(flight_offers_response)
        if flight_data_table is not None:
            return flight_data_table
    else:
        print("No flight offers found.")
        return None

In [9]:
def fetch_10_day_forecast(city, api_key):
    '''
    Function to fetch the 10-day weather forecast for a given city.
    
    Parameters:
    - city: The name of the city for which the forecast is requested.
    - api_key: The API key required for accessing the weather forecast API.
    
    Returns:
    - A pandas DataFrame containing the extracted forecast data.
    '''
    # Construct the base URL for the weather forecast API
    base_url = 'http://api.weatherapi.com/v1'
    
    # Construct the complete URL with the API key, city, and number of days for forecast
    url = f"{base_url}/forecast.json?key={api_key}&q={city}&days=10"
    
    # Send a GET request to the API endpoint
    response = requests.get(url)
    
    # Parse the JSON response into a Python dictionary
    data = response.json()
    
    # Check if the response contains the necessary forecast data
    if 'forecast' in data and 'forecastday' in data['forecast']:
        # Extract the forecast data for each day
        forecast_data = data['forecast']['forecastday']
        
        # Initialize an empty list to store the extracted data for each day
        extracted_data = []
        
        # Iterate over each day's forecast data
        for day in forecast_data:
            # Extract relevant information such as date, temperature, humidity, etc.
            date = day['date']
            temp = day['day']['avgtemp_c']
            humidity = day['day']['avghumidity']
            cloudiness = day['day']['condition']['text']
            wind_speed = day['day']['maxwind_kph']
            
            # Append the extracted data for the current day to the list
            extracted_data.append({
                'city': city,
                'date': date,
                'temp': temp,
                'humidity': humidity,
                'condition': cloudiness,
                'wind_speed': wind_speed
            })
        
        # Convert the list of dictionaries into a pandas DataFrame
        return pd.DataFrame(extracted_data)
    else:
        # Print a message if no forecast data is available for the specified city
        print(f"No forecast data available for {city}")
        
        # Return an empty DataFrame
        return pd.DataFrame()



The previous function used a weather forecast API to return the forecast for the next 10 days. Ideally, with a subscription API we would be able to get the forecast for the following 10 days from the selected date of departure.

In [ ]:
def on_update_button_clicked(button):
    '''
    Function to handle the click event of the update button.
    
    Parameters:
    - button: The button widget that triggers the event.
    '''
    # Retrieve the selected city from the city dropdown widget
    city = city_select.value
    
    # Retrieve the access token for accessing flight details API
    access_token = get_access_token()
    
    # Set the origin location code to 'MAD' (assuming Madrid as the origin airport)
    originLocationCode = 'MAD'
    
    # Retrieve the selected destination airport code from the airport dropdown widget
    destinationLocationCode = airport_select.value
    
    # Retrieve the selected departure date from the date picker widget and format it
    departure_date = departure_date_picker.value.strftime('%Y-%m-%d')
    
    # Clear any previous outputs in the flight details and weather output widgets
    flight_details_output.clear_output()
    weather_output.clear_output()
    
    with flight_details_output:
        # Fetch and display flight details based on the selected city, access token, origin and destination codes, and departure date
        flight_data_table = update_flight_offers(city, access_token, originLocationCode, destinationLocationCode, departure_date)
        
        # Check if flight data table is not None
        if flight_data_table is not None:
            # Display the flight data table
            display(flight_data_table)
        else:
            # Print a message if no flight offers are found
            print("No flight offers found.")
    
    with weather_output:
        # Fetch and display weather forecast for the selected city
        # Replace 'weather_key' with your actual API key
        forecast_df = fetch_10_day_forecast(city, weather_key)
        
        # Check if forecast DataFrame is not empty
        if not forecast_df.empty:
            # Display the weather forecast DataFrame
            display(forecast_df)
        else:
            # Print a message if no forecast data is available
            print("No forecast data available.")


In [50]:
# Initialize Bokeh for output in the notebook
output_notebook()

# List of cities
cities = ['Madrid', 'London', 'Paris', 'Barcelona']

# Create widgets
city_select = widgets.Dropdown(options=cities, description='Select City')
airport_select = widgets.Dropdown(description='Select Airport')
departure_date_picker = widgets.DatePicker(description='Select Date')
update_button = widgets.Button(description='Get Flights')
flight_details_output = widgets.Output()  # Output widget to display flight details table
weather_output = widgets.Output()  # Output widget to display weather forecast



city_select.observe(update_airport_options, names='value')

# Attach the event handler to the update button
update_button.on_click(on_update_button_clicked)

# Define a container for the widgets
inputs = VBox([city_select, airport_select, departure_date_picker, update_button])

# Display the widgets and outputs
display(inputs, flight_details_output, weather_output)

Loading BokehJS ...

Output()

Output()